In [1]:
import numpy as np
import scipy.io as spio
from matplotlib import pyplot as plt
from matplotlib import cm
import glob
import os
import tensorflow as tf
from util import constants
from util.config_util import get_model_params, get_task_params, get_train_params
from tf2_models.trainer import Trainer
from absl import app
from absl import flags
import numpy as np
from util.models import MODELS
from util.tasks import TASKS
import tensorflow_probability as tfp
import tensorflow_datasets as tfds
from tfds_data.aff_nist import AffNist
from calibration_util import *
from notebook_utils import *
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()
sns.set_style("whitegrid")

from tqdm import tqdm

def visualization(x, y,count,index):
    x = np.reshape(x, (40, 40))
    
    plt.subplot(1,count,index)
    plt.imshow(x, cmap=cm.Greys_r)
    plt.title(y)
    plt.axis('off')   

def visualization_overlap(x0,x1, y0,y1,count,index):
    r = np.reshape(x0, (40, 40))
    g = np.reshape(x1, (40, 40))
    b = np.zeros_like(r)
    rgb = np.stack([r,g,b],-1)
    
    plt.subplot(1,count,index)
    plt.imshow(rgb)
    plt.title('R:('+ str(y0)+','+str(y1)+')')
    plt.axis('off')   


[nltk_data] Downloading package punkt to /home/dehghani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
chkpt_dir='../tf_ckpts'
task_name = 'mnist'
mnist = TASKS[task_name](get_task_params(), data_dir='../data')

chkpt_dir='../tf_ckpts'
task_name = 'affnist'
affnist = TASKS[task_name](get_task_params(), data_dir='../data')

In [3]:
def convert_examples(examples):
    pad_length = int((28 - 28) / 2)
    return tf.pad(tf.cast(examples['image'], dtype=tf.float32) / 255,
                  ([pad_length, pad_length], [pad_length, pad_length],
                   [0, 0])), tf.cast(
      examples['label'], dtype=tf.int32)

cmnist_trans = tfds.builder('mnist_corrupted/translate')
cmnist_trans_test = cmnist_trans.as_dataset(split='test')
cmnist_trans_test = cmnist_trans_test.map(map_func=lambda x: convert_examples(x))
cmnist_trans_test = cmnist_trans_test.batch(128)
cmnist_trans.info.splits['test'].num_examples


cmnist_trans_train = cmnist_trans.as_dataset(split='train')
cmnist_trans_train = cmnist_trans_train.map(map_func=lambda x: convert_examples(x))
cmnist_trans_train = cmnist_trans_train.batch(128)
cmnist_trans_train = cmnist_trans_train.repeat(100)

cmnist_trans.info.splits['train'].num_examples / 128

468.75

In [4]:
cmnist_scale = tfds.builder('mnist_corrupted/scale')
cmnist_scale_test = cmnist_scale.as_dataset(split='test')
cmnist_scale_test = cmnist_scale_test.map(map_func=lambda x: convert_examples(x))
cmnist_scale_test = cmnist_scale_test.batch(128)
cmnist_scale.info.splits['test'].num_examples

cmnist_scale_train = cmnist_scale.as_dataset(split='train')
cmnist_scale_train = cmnist_scale_train.map(map_func=lambda x: convert_examples(x))
cmnist_scale_train = cmnist_scale_train.batch(128)
cmnist_scale_train = cmnist_scale_train.repeat(100)

cmnist_scale.info.splits['train'].num_examples

60000

In [ ]:
count = 10
for x,y in affnist.test_dataset:
    print(x.shape)
    #visualization(x[1].numpy(),y[1].numpy(),1,1)
    plt.imshow(x[0,...,0])
    plt.show()
    count -= 1
    if count <= 0:
        break

In [ ]:
count = 10
for x,y in mnist.valid_dataset:
    print(x.shape)
    #visualization(x[1].numpy(),y[1].numpy(),1,1)
    plt.imshow(x[0,...,0])
    plt.show()
    count -= 1
    if count <= 0:
        break

In [5]:
config={'exp_name':'test',
    'model_config':'ff_mnist4',
    'task_name':'mnist',
    'model_name':'cl_vff',
    'chkpt_dir':'../tf_ckpts',
    'learning_rate': 0.001
    }

task = TASKS[config['task_name']](get_task_params(batch_size=128), data_dir='../data')

hparams = get_model_params(task, config['model_name'], config['model_config'])
print(hparams, hparams.__dict__)

#model, _ = get_model(config, task, hparams, 0)

model config: ff_mnist4
<util.model_configs.ModelConfig object at 0x7f4264363790> {'embedding_dim': 512, 'hidden_dim': [512, 128, 32], 'input_dim': 784, 'output_dim': 10, 'depth': 3, 'proj_depth': 1, 'fc_dim': [], 'hidden_dropout_rate': 0.1, 'input_dropout_rate': 0.0, 'initializer_range': None, 'kernel_size': [(3, 3)], 'filters': [32], 'maxout_size': [32], 'pool_size': [(2, 2)], 'output_hidden_states': False, 'output_embeddings': False, 'routings': 3}


In [6]:
class VanillaFF(tf.keras.models.Sequential):
  def __init__(self, hparams, scope="cl_vff", *inputs, **kwargs):
    if 'cl_token' in kwargs:
      del kwargs['cl_token']

    super(VanillaFF, self).__init__()
    self.scope = scope
    self.hparams = hparams

    self.model_name = '_'.join([self.scope,
                                'h-' + '.'.join([str(x) for x in self.hparams.hidden_dim]),
                                'd-' + str(self.hparams.depth),
                                'hdrop-' + str(self.hparams.hidden_dropout_rate),
                                'indrop-' + str(self.hparams.input_dropout_rate)])

    self.regularizer = tf.keras.regularizers.l1_l2(l1=0.00001,
                                                   l2=0.00001)
    self.create_vars()
    self.rep_index = 1
    self.rep_layer = -1



  def create_vars(self):
    self.flat = tf.keras.layers.Flatten()
    # self.batch_norm = tf.keras.layers.BatchNormalization()
    # self.batch_norm.trainable = True
    self.indrop = tf.keras.layers.Dropout(self.hparams.input_dropout_rate)
    self.activation = tf.keras.layers.Activation('relu')

    self.hidden_layers = []
    self.hidden_batch_norms = []
    self.hidden_dropouts = []
    for i in np.arange(self.hparams.depth):
      self.hidden_layers.append(tf.keras.layers.Dense(self.hparams.hidden_dim[i],
                                     activation=None, #'relu',
                                     kernel_regularizer=self.regularizer))
      #self.hidden_batch_norms.append(tf.keras.layers.BatchNormalization())
      #self.hidden_batch_norms[i].trainable = True
      self.hidden_dropouts.append(tf.keras.layers.Dropout(self.hparams.hidden_dropout_rate))

    self.final_dense = tf.keras.layers.Dense(self.hparams.output_dim,
                                   kernel_regularizer=self.regularizer)


  def call(self, inputs, padding_symbol=None, training=None, **kwargs):
    x = self.flat(inputs, **kwargs)
    # x = self.batch_norm(x, training=training, **kwargs)
    x = self.indrop(x, training=training, **kwargs)

    for i in np.arange(self.hparams.depth):
      x = self.hidden_layers[i](x, training=training, **kwargs)
      x = self.activation(x)
      #x = self.hidden_batch_norms[i](x, training=training, **kwargs)
      x = self.hidden_dropouts[i](x, training=training, **kwargs)

    logits = self.final_dense(x, training=training, **kwargs)

    return logits


  def detailed_call(self, inputs, padding_symbol=None, training=None, **kwargs):
    layer_activations = []
    x = self.flat(inputs, **kwargs)
    x = self.indrop(x, training=None, **kwargs)
    layer_activations.append(x)

    for i in np.arange(self.hparams.depth):
      x = self.hidden_layers[i](x, training=training, **kwargs)
      x = self.activation(x)
      x = self.hidden_batch_norms[i](x, training=training, **kwargs)
      x = self.hidden_dropouts[i](x, training=training, **kwargs)
      layer_activations.append(x)

    pnltimt = x
    logits = self.final_dense(x, training=None, **kwargs)

    return logits, pnltimt, layer_activations

In [7]:
# hparams.depth=4
# hparams.hidden_dims=[512, 512, 64, 32]
# hparams.hidden_dropout_rate = 0.2
hparams.fc_dim = []
hparams.depth = 3
hparams.hidden_dims = [128, 64, 64]
hparams.hidden_dropout_rate =  0.3
hparams.input_dropout_rate = 0.0
model = VanillaFF(hparams)
for x,y in task.train_dataset:
    print(x.shape, y.shape)
    break
out = model(inputs=x)
print(out.shape)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=task.get_loss_fn(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.summary()

(128, 28, 28, 1) (128,)
(128, 10)
Model: "vanilla_ff"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dropout (Dropout)            multiple                  0         
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  401920    
_________________________________________________________________
dense_1 (Dense)              multiple                  65664     
_________________________________________________________________
dense_2 (Dense)              multiple                  4128      
_________________________________________________________________
dropout_1 (Dropout)   

In [8]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=task.get_loss_fn(),
              metrics=task.metrics())

history = model.fit(task.train_dataset,
                  epochs=100,
                  steps_per_epoch=task.n_train_batches,
                  validation_data=task.test_dataset,
                  validation_steps=task.n_test_batches,
                  verbose=2
                  )



TypeError: Unrecognized keyword arguments: {'test_steps': 468, 'test_data': <DatasetV1Adapter shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int32)>}

In [ ]:
model.evaluate(task.test_dataset, steps=task.n_test_batches)

In [ ]:
model.evaluate(cmnist_trans_test, steps=cmnist_trans.info.splits['test'].num_examples/128)

In [ ]:
model.evaluate(cmnist_scale_test, steps=cmnist_scale.info.splits['test'].num_examples/128)

In [ ]:
import tensorflow_datasets as tfds


In [11]:
history = model.fit(task.train_dataset,
                  epochs=1,
                  steps_per_epoch=task.n_train_batches)
mnist = model.evaluate(task.test_dataset, steps=task.n_test_batches)
trans = model.evaluate(cmnist_trans_test, steps=cmnist_trans.info.splits['test'].num_examples/128)
scale = model.evaluate(cmnist_scale_test, steps=cmnist_scale.info.splits['test'].num_examples/128)

print("mnist:", mnist)
print("trans:", trans)
print("scale:", scale)

Train for 468 steps
79/78 [==============================] - 0s 3ms/step - loss: 1.8512 - classification_loss: 1.7082 - sparse_categorical_accuracy: 0.3928
mnist: [0.4186604210199454, 0.27587843, 0.9244792]
trans: [3.9046855812072754, 3.7186553, 0.1197]
scale: [1.8717201343536376, 1.7082068, 0.3928]


In [ ]:
tfds.load('mnist_corrupted/scale', split='test')
tfds.load('mnist_corrupted/translate', split='test')